### ENET FOR IMAGES

In [2]:
# import the necessary packages
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [ ]:
start = time.time()
SET_WIDTH = int(600)

normalize_image = 1 / 255.0
resize_image_shape = (1024, 512)

sample_img = cv2.imread('./images/example_02.jpg')
sample_img = imutils.resize(sample_img, width=SET_WIDTH)

blob_img = cv2.dnn.blobFromImage(sample_img, normalize_image, resize_image_shape, 0,swapRB=True, crop=False)

In [ ]:
blob_img

In [ ]:
print("[INFO] loading model...")
cv_enet_model = cv2.dnn.readNet('./enet-cityscapes/enet-model.net')

In [ ]:
cv_enet_model.setInput(blob_img)

cv_enet_model_output = cv_enet_model.forward()

In [ ]:
label_values = open('./enet-cityscapes/enet-classes.txt').read().strip().split("\n")

In [ ]:
IMG_OUTPUT_SHAPE_START =1
IMG_OUTPUT_SHAPE_END =4
(classes_num, h, w) = cv_enet_model_output.shape[IMG_OUTPUT_SHAPE_START:IMG_OUTPUT_SHAPE_END]

In [ ]:
class_map = np.argmax(cv_enet_model_output[0], axis=0)

In [ ]:
import os
if os.path.isfile('./enet-cityscapes/enet-colors.txt'):
    CV_ENET_SHAPE_IMG_COLORS = open('./enet-cityscapes/enet-colors.txt').read().strip().split("\n")
    CV_ENET_SHAPE_IMG_COLORS = [np.array(c.split(",")).astype("int") for c in CV_ENET_SHAPE_IMG_COLORS]
    CV_ENET_SHAPE_IMG_COLORS = np.array(CV_ENET_SHAPE_IMG_COLORS, dtype="uint8")

else:
   
    np.random.seed(42)
    CV_ENET_SHAPE_IMG_COLORS = np.random.randint(0, 255, size=(len(label_values) - 1, 3),
                               dtype="uint8")
    CV_ENET_SHAPE_IMG_COLORS = np.vstack([[0, 0, 0], CV_ENET_SHAPE_IMG_COLORS]).astype("uint8")

In [ ]:
mask_class_map = CV_ENET_SHAPE_IMG_COLORS[class_map]

In [ ]:
mask_class_map = cv2.resize(mask_class_map, (sample_img.shape[1], sample_img.shape[0]),
                  interpolation=cv2.INTER_NEAREST)

class_map = cv2.resize(mask_class_map, (sample_img.shape[1], sample_img.shape[0]),
                      interpolation=cv2.INTER_NEAREST)

In [ ]:
cv_enet_model_output = ((0.4 * sample_img) + (0.6 * mask_class_map)).astype("uint8")

In [ ]:
my_legend = np.zeros(((len(label_values) * 25) + 25, 300, 3), dtype="uint8")

In [ ]:
for (i, (class_name, img_color)) in enumerate(zip(label_values, CV_ENET_SHAPE_IMG_COLORS)):
    # draw the class name + color on the legend
    color_info = [int(color) for color in img_color]
    cv2.putText(my_legend, class_name, (5, (i * 25) + 17),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.rectangle(my_legend, (100, (i * 25)), (300, (i * 25) + 25),
                  tuple(color_info), -1)

In [ ]:
cv2.imshow("My_Legend", my_legend)
cv2.imshow("Img_Input", sample_img)
cv2.imshow("CV_Model_Output", cv_enet_model_output)
cv2.waitKey(0)
cv2.destroyAllWindows()

end = time.time()
print("[INFO] inference took {:.4f} seconds".format(end - start))


### ENET FOR VIDEOS

In [12]:
import os
import time
import cv2
import imutils
import numpy as np


DEFAULT_FRAME = 1
SET_WIDTH = 600

In [13]:
class_labels = open('./enet-cityscapes/enet-classes.txt').read().strip().split("\n")

In [14]:
if os.path.isfile('./enet-cityscapes/enet-colors.txt'):
    CV_ENET_SHAPE_IMG_COLORS = open('./enet-cityscapes/enet-colors.txt').read().strip().split("\n")
    CV_ENET_SHAPE_IMG_COLORS = [np.array(c.split(",")).astype("int") for c in CV_ENET_SHAPE_IMG_COLORS]
    CV_ENET_SHAPE_IMG_COLORS = np.array(CV_ENET_SHAPE_IMG_COLORS, dtype="uint8")

else:
    
    np.random.seed(42)
    CV_ENET_SHAPE_IMG_COLORS = np.random.randint(0, 255, size=(len(class_labels) - 1, 3),
                               dtype="uint8")
    CV_ENET_SHAPE_IMG_COLORS = np.vstack([[0, 0, 0], CV_ENET_SHAPE_IMG_COLORS]).astype("uint8")

In [15]:
print("[INFO] loading model...")
cv_enet_model = cv2.dnn.readNet('./enet-cityscapes/enet-model.net')

[INFO] loading model...


In [16]:
sv = cv2.VideoCapture('.//video//video.mp4')
sample_video_writer = None

In [17]:
print(sv)

<VideoCapture 000002378F527EF0>


In [18]:
prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT

In [19]:
total = int(sv.get(prop))

In [20]:
total

1007

In [ ]:

# try to determine the total number of frames in the video file
try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT
    total = int(sv.get(prop))
    print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
    print("[INFO] could not determine # of frames in video")
    total = -1
#sample_video
# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = sv.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # construct a blob from the frame and perform a forward pass
    # using the segmentation model
    normalize_image = 1 / 255.0
    resize_image_shape = (1024, 512)
    video_frame = imutils.resize(frame, width=SET_WIDTH)
    blob_img = cv2.dnn.blobFromImage(frame,  normalize_image,resize_image_shape, 0,
                                 swapRB=True, crop=False)
    cv_enet_model.setInput(blob_img)
    start = time.time()
    cv_enet_model_output = cv_enet_model.forward()
    end = time.time()

    # infer the total number of classes along with the spatial
    # dimensions of the mask image via the shape of the output array
    (Classes_num, height, width) = cv_enet_model_output.shape[1:4]

    # our output class ID map will be num_classes x height x width in
    # size, so we take the argmax to find the class label with the
    # largest probability for each and every (x, y)-coordinate in the
    # image
    classMap = np.argmax(cv_enet_model_output[0], axis=0)

    # given the class ID map, we can map each of the class IDs to its
    # corresponding color
    
    mask_class_map = CV_ENET_SHAPE_IMG_COLORS[classMap]

    # resize the mask such that its dimensions match the original size
    # of the input frame
    
    
    mask_class_map = cv2.resize(mask_class_map, (video_frame.shape[1], video_frame.shape[0]),
                      interpolation=cv2.INTER_NEAREST)

    # perform a weighted combination of the input frame with the mask
    # to form an output visualization
    
    
    cv_enet_model_output = ((0.3 * video_frame) + (0.7 * mask_class_map)).astype("uint8")

    # check if the video writer is None
    if sample_video_writer is None:
        print("sample_video_writer is None")
        # initialize our video writer
        fourcc_obj = cv2.VideoWriter_fourcc(*"MJPG")

        sample_video_writer = cv2.VideoWriter('./output/output_toronoto.avi', fourcc_obj, 30,
                                 (cv_enet_model_output.shape[1], cv_enet_model_output.shape[0]), True)

        # some information on processing single frame
        if total > 0:
            
            execution_time = (end - start)
            print("[INFO] single video_frame took {:.4f} seconds".format(execution_time))

            print("[INFO] estimated total_time time: {:.4f}".format(
                execution_time * total))

    # write the output frame to disk
    
    sample_video_writer.write(cv_enet_model_output)

    # check to see if we should display the output frame to our screen
    if DEFAULT_FRAME > 0:
        cv2.imshow("Video Frame", cv_enet_model_output)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord("q"):
            break

print("[INFO] cleaning up...")
sample_video_writer.release()
sv.release()

[INFO] 1007 total frames in video
(True, array([[[227, 232, 225],
        [225, 230, 223],
        [235, 227, 217],
        ...,
        [228, 212, 197],
        [228, 212, 197],
        [228, 212, 197]],

       [[227, 232, 225],
        [225, 230, 223],
        [235, 227, 217],
        ...,
        [228, 212, 197],
        [228, 212, 197],
        [228, 212, 197]],

       [[228, 233, 226],
        [225, 230, 223],
        [234, 226, 216],
        ...,
        [228, 212, 197],
        [228, 212, 197],
        [228, 212, 197]],

       ...,

       [[ 58,  48,  48],
        [ 56,  46,  46],
        [ 56,  46,  46],
        ...,
        [109, 106, 107],
        [112, 109, 110],
        [103, 100, 101]],

       [[ 54,  44,  44],
        [ 54,  44,  44],
        [ 55,  45,  45],
        ...,
        [109, 106, 107],
        [123, 120, 121],
        [ 96,  93,  94]],

       [[ 51,  41,  41],
        [ 53,  43,  43],
        [ 55,  45,  45],
        ...,
        [116, 113, 114],
        

(True, array([[[196, 177, 158],
        [196, 177, 158],
        [196, 177, 158],
        ...,
        [228, 212, 197],
        [228, 212, 197],
        [228, 212, 197]],

       [[196, 177, 158],
        [196, 177, 158],
        [197, 178, 159],
        ...,
        [228, 212, 197],
        [228, 212, 197],
        [228, 212, 197]],

       [[197, 178, 159],
        [197, 178, 159],
        [197, 178, 159],
        ...,
        [228, 212, 197],
        [228, 212, 197],
        [228, 212, 197]],

       ...,

       [[ 56,  46,  46],
        [ 55,  45,  45],
        [ 55,  45,  45],
        ...,
        [109, 106, 107],
        [118, 115, 116],
        [ 98,  95,  96]],

       [[ 56,  46,  46],
        [ 55,  45,  45],
        [ 55,  45,  45],
        ...,
        [ 98,  95,  96],
        [118, 115, 116],
        [102,  99, 100]],

       [[ 56,  46,  46],
        [ 55,  45,  45],
        [ 55,  45,  45],
        ...,
        [102,  99, 100],
        [138, 135, 136],
        [118, 115

(True, array([[[195, 176, 157],
        [195, 176, 157],
        [195, 176, 157],
        ...,
        [229, 213, 198],
        [229, 213, 198],
        [229, 213, 198]],

       [[196, 177, 158],
        [195, 176, 157],
        [195, 176, 157],
        ...,
        [229, 213, 198],
        [229, 213, 198],
        [229, 213, 198]],

       [[196, 177, 158],
        [196, 177, 158],
        [196, 177, 158],
        ...,
        [229, 213, 198],
        [229, 213, 198],
        [229, 213, 198]],

       ...,

       [[ 30,  16,  14],
        [ 30,  16,  14],
        [ 30,  16,  14],
        ...,
        [ 60,  46,  44],
        [ 57,  43,  41],
        [ 64,  50,  48]],

       [[ 30,  16,  14],
        [ 30,  16,  14],
        [ 30,  16,  14],
        ...,
        [ 55,  44,  41],
        [ 55,  44,  41],
        [ 60,  49,  46]],

       [[ 29,  15,  13],
        [ 29,  15,  13],
        [ 29,  15,  13],
        ...,
        [ 50,  39,  36],
        [ 58,  47,  44],
        [ 61,  50

(True, array([[[197, 176, 157],
        [197, 176, 157],
        [197, 176, 157],
        ...,
        [227, 211, 196],
        [229, 214, 196],
        [229, 214, 196]],

       [[197, 176, 157],
        [197, 176, 157],
        [197, 176, 157],
        ...,
        [228, 212, 197],
        [231, 216, 198],
        [231, 216, 198]],

       [[197, 176, 157],
        [197, 176, 157],
        [197, 176, 157],
        ...,
        [228, 212, 197],
        [229, 213, 198],
        [229, 213, 198]],

       ...,

       [[ 50,  38,  40],
        [ 50,  38,  40],
        [ 50,  38,  40],
        ...,
        [ 79,  73,  76],
        [108, 101, 107],
        [109, 102, 108]],

       [[ 50,  38,  40],
        [ 50,  38,  40],
        [ 51,  39,  41],
        ...,
        [ 72,  66,  69],
        [107, 100, 106],
        [118, 111, 117]],

       [[ 50,  38,  40],
        [ 49,  37,  39],
        [ 51,  39,  41],
        ...,
        [ 77,  71,  74],
        [114, 107, 113],
        [130, 123

(True, array([[[196, 175, 156],
        [196, 175, 156],
        [196, 175, 156],
        ...,
        [227, 209, 194],
        [229, 211, 196],
        [231, 213, 198]],

       [[196, 175, 156],
        [196, 175, 156],
        [196, 175, 156],
        ...,
        [227, 209, 194],
        [230, 212, 197],
        [231, 213, 198]],

       [[196, 175, 156],
        [196, 175, 156],
        [196, 175, 156],
        ...,
        [228, 210, 195],
        [231, 213, 198],
        [231, 213, 198]],

       ...,

       [[ 44,  34,  34],
        [ 44,  34,  34],
        [ 46,  36,  36],
        ...,
        [ 90,  85,  84],
        [ 87,  82,  81],
        [ 82,  77,  76]],

       [[ 44,  34,  34],
        [ 44,  34,  34],
        [ 45,  35,  35],
        ...,
        [ 80,  72,  74],
        [103,  95,  97],
        [ 87,  79,  81]],

       [[ 44,  34,  34],
        [ 44,  34,  34],
        [ 45,  35,  35],
        ...,
        [ 77,  69,  71],
        [122, 114, 116],
        [109, 101

...

(True, array([[[189, 168, 149],
        [190, 169, 150],
        [190, 169, 150],
        ...,
        [233, 217, 202],
        [235, 219, 204],
        [235, 219, 204]],

       [[189, 168, 149],
        [190, 169, 150],
        [190, 169, 150],
        ...,
        [233, 217, 202],
        [234, 218, 203],
        [235, 219, 204]],

       [[190, 169, 150],
        [190, 169, 150],
        [190, 169, 150],
        ...,
        [232, 216, 201],
        [234, 218, 203],
        [234, 218, 203]],

       ...,

       [[ 47,  37,  37],
        [ 48,  38,  38],
        [ 49,  39,  39],
        ...,
        [126, 121, 120],
        [101,  96,  95],
        [ 85,  80,  79]],

       [[ 47,  37,  37],
        [ 48,  38,  38],
        [ 48,  38,  38],
        ...,
        [116, 111, 110],
        [107, 102, 101],
        [104,  99,  98]],

       [[ 47,  37,  37],
        [ 46,  36,  36],
        [ 47,  37,  37],
        ...,
        [118, 113, 112],
        [ 95,  90,  89],
        [122, 117

(True, array([[[189, 168, 149],
        [188, 167, 148],
        [190, 169, 150],
        ...,
        [234, 218, 203],
        [232, 216, 201],
        [234, 218, 203]],

       [[189, 168, 149],
        [188, 167, 148],
        [190, 169, 150],
        ...,
        [233, 217, 202],
        [233, 217, 202],
        [236, 220, 205]],

       [[189, 168, 149],
        [188, 167, 148],
        [190, 169, 150],
        ...,
        [234, 218, 203],
        [233, 217, 202],
        [234, 218, 203]],

       ...,

       [[ 45,  35,  35],
        [ 45,  35,  35],
        [ 46,  36,  36],
        ...,
        [ 95,  92,  91],
        [ 84,  81,  80],
        [ 93,  90,  89]],

       [[ 45,  35,  35],
        [ 45,  35,  35],
        [ 45,  35,  35],
        ...,
        [103, 100,  99],
        [104, 101, 100],
        [117, 114, 113]],

       [[ 45,  35,  35],
        [ 45,  35,  35],
        [ 45,  35,  35],
        ...,
        [112, 109, 108],
        [110, 107, 106],
        [124, 121

(True, array([[[191, 167, 149],
        [191, 167, 149],
        [193, 169, 151],
        ...,
        [234, 217, 204],
        [234, 217, 204],
        [235, 218, 205]],

       [[191, 167, 149],
        [191, 167, 149],
        [193, 169, 151],
        ...,
        [234, 217, 204],
        [234, 217, 204],
        [235, 218, 205]],

       [[190, 166, 148],
        [191, 167, 149],
        [192, 168, 150],
        ...,
        [233, 216, 203],
        [235, 218, 205],
        [235, 218, 205]],

       ...,

       [[ 51,  41,  41],
        [ 51,  41,  41],
        [ 52,  42,  42],
        ...,
        [136, 131, 132],
        [115, 110, 111],
        [ 89,  84,  85]],

       [[ 48,  38,  38],
        [ 49,  39,  39],
        [ 51,  41,  41],
        ...,
        [132, 127, 128],
        [111, 106, 107],
        [ 93,  88,  89]],

       [[ 46,  36,  36],
        [ 47,  37,  37],
        [ 48,  38,  38],
        ...,
        [125, 120, 121],
        [ 99,  94,  95],
        [102,  97

(True, array([[[190, 166, 148],
        [192, 168, 150],
        [193, 169, 151],
        ...,
        [235, 219, 204],
        [234, 218, 203],
        [233, 217, 202]],

       [[191, 167, 149],
        [192, 168, 150],
        [193, 169, 151],
        ...,
        [234, 218, 203],
        [234, 218, 203],
        [235, 219, 204]],

       [[189, 168, 149],
        [190, 169, 150],
        [191, 170, 151],
        ...,
        [233, 217, 202],
        [234, 218, 203],
        [235, 219, 204]],

       ...,

       [[ 47,  39,  39],
        [ 49,  41,  41],
        [ 49,  41,  41],
        ...,
        [110, 107, 106],
        [120, 117, 116],
        [104, 101, 100]],

       [[ 47,  39,  39],
        [ 47,  39,  39],
        [ 49,  41,  41],
        ...,
        [111, 108, 109],
        [145, 142, 143],
        [102,  99, 100]],

       [[ 45,  37,  37],
        [ 46,  38,  38],
        [ 46,  38,  38],
        ...,
        [128, 125, 126],
        [153, 150, 151],
        [102,  99

(True, array([[[189, 168, 149],
        [189, 168, 149],
        [189, 168, 149],
        ...,
        [236, 220, 205],
        [236, 220, 205],
        [235, 219, 204]],

       [[190, 169, 150],
        [190, 169, 150],
        [190, 169, 150],
        ...,
        [235, 219, 204],
        [235, 219, 204],
        [234, 218, 203]],

       [[189, 168, 149],
        [189, 168, 149],
        [189, 168, 149],
        ...,
        [234, 218, 203],
        [233, 217, 202],
        [232, 216, 201]],

       ...,

       [[ 49,  39,  39],
        [ 51,  41,  41],
        [ 51,  41,  41],
        ...,
        [100, 100, 100],
        [ 86,  86,  86],
        [160, 160, 160]],

       [[ 47,  37,  37],
        [ 51,  41,  41],
        [ 51,  41,  41],
        ...,
        [ 94,  94,  94],
        [108, 108, 108],
        [156, 156, 156]],

       [[ 48,  38,  38],
        [ 51,  41,  41],
        [ 49,  39,  39],
        ...,
        [ 86,  86,  86],
        [135, 135, 135],
        [146, 146